<a href="https://colab.research.google.com/github/princexoleo/news_scrapping_and_ranking_similarity/blob/master/News_ranking_with_cosine_similarities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#News Scrapping from Various bangla Newspaper


*   [মানবজমিন](https://www.mzamin.com/)



##Import Necessary Libraries

In [0]:
from requests import get 
from bs4 import BeautifulSoup
import pandas as pd

##Implement Scrapping Function

In [0]:
def scrapping_from_newspaper(newspaper_url, domain_newspaper):
  # initialize some temporary list
  print("[info] Newspaper Scarpping started ..")
  all_url = []
  all_titles = []
  all_contents = []

  # get html from that given newspaper url
  news_html = get(newspaper_url)
  # create a beautifulsoup object from that news_html page
  soup = BeautifulSoup(news_html.text,'html.parser')
  # find <li> tag from the soup objects
  href_containers = soup.find_all('li',class_="details-list")
  # now iterate to get the news title and url for details of the news
  for ob in href_containers:
    # find <a> tag
    temp_a = ob.find('a')
    # append the new title 
    all_titles.append(temp_a.text)
    # concate domainName with temp_a.get(href) which will help us to get the full details of the news
    # like "www.something.com/"+"article/157123-sometitle"
    deep_url = domain_newspaper+temp_a.get('href')
    # append in a list
    all_url.append(deep_url)
    # now get the full details/content of the news from deep_url
    deep_html = get(deep_url)
    deep_soup = BeautifulSoup(deep_html.text, "html.parser")
    deep_content = deep_soup.find('div', class_="details-text")
    # and append in a list
    all_contents.append(deep_content)
  
  # finally return every thing in pandas csv format
  df = pd.DataFrame()
  df["URL"] = all_url.copy()
  df["Title"] = all_titles.copy()
  df["Details"] = all_contents.copy()
  print("[info] scrapping news succesfully done..")
  
  return df

##Calling NewsScrapping Function

In [17]:
# news paper link which we want to scrap
##
url = "https://www.mzamin.com/archive-result.php?data=1587146400&date=2020-04-18"
main_domain ="https://www.mzamin.com/"
## call the function
df = scrapping_from_newspaper(url, main_domain)
df.head()

[info] Newspaper Scarpping started ..
[info] scrapping news succesfully done..


,URL,Title,Details
0,https://www.mzamin.com/article.php?mzamin=2225...,"ইইউ ক্যাপিটালে মৃত্যুর মিছিল, এখনও নিরাপদে আছে...","[\n, [\n, [মিজানুর রহমান], \n, [<h3>অনলাইন <sm..."
1,https://www.mzamin.com/article.php?mzamin=2225...,করোনা কেড়ে নিলো দেড় লাখ প্রাণ,"[\n, [\n, [অনলাইন ডেস্ক ], \n, [<h3>অনলাইন <sm..."
2,https://www.mzamin.com/article.php?mzamin=2225...,‘আমিই ভেন্টিলেটর বন্ধ করছি আর রোগীর শেষ মুহূর্...,"[\n, [\n, [], \n, [<h3>অনলাইন <smal..."
3,https://www.mzamin.com/article.php?mzamin=2225...,আরো কঠোর হচ্ছে পুলিশ,"[\n, [\n, [স্টাফ রিপোর্টার], \n, [<h3>অনলাইন <..."
4,https://www.mzamin.com/article.php?mzamin=2225...,আইনমন্ত্রীর মায়ের ইন্তেকাল,"[\n, [\n, [ অনলাইন ডেস্ক], \n, [<h3>অনলাইন <sm..."


#Ranking News with Cosine Algorithm

In [0]:
# Import Libraries 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
#data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/NewsRankingTool/news_april18.csv") 
#data.head()
data = df

In [27]:
# clearning dataset: drop index column
data = data.drop('URL', axis=1) # URL column dropped
data.head()

,Title,Details
0,"ইইউ ক্যাপিটালে মৃত্যুর মিছিল, এখনও নিরাপদে আছে...","[\n, [\n, [মিজানুর রহমান], \n, [<h3>অনলাইন <sm..."
1,করোনা কেড়ে নিলো দেড় লাখ প্রাণ,"[\n, [\n, [অনলাইন ডেস্ক ], \n, [<h3>অনলাইন <sm..."
2,‘আমিই ভেন্টিলেটর বন্ধ করছি আর রোগীর শেষ মুহূর্...,"[\n, [\n, [], \n, [<h3>অনলাইন <smal..."
3,আরো কঠোর হচ্ছে পুলিশ,"[\n, [\n, [স্টাফ রিপোর্টার], \n, [<h3>অনলাইন <..."
4,আইনমন্ত্রীর মায়ের ইন্তেকাল,"[\n, [\n, [ অনলাইন ডেস্ক], \n, [<h3>অনলাইন <sm..."


In [28]:
input_text = data.loc[0,:]
print(str(input_text))

Title      ইইউ ক্যাপিটালে মৃত্যুর মিছিল, এখনও নিরাপদে আছে...
Details    [\n, [\n, [মিজানুর রহমান], \n, [<h3>অনলাইন <sm...
Name: 0, dtype: object


In [30]:
# load files from drive/pc 
# bangla stop words
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving bangla_stopwords.txt to bangla_stopwords.txt
User uploaded file "bangla_stopwords.txt" with length 5840 bytes


In [33]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt') # if necessary

with open('/content/bangla_stopwords.txt', 'r') as infile:
   stop_words = infile.read().splitlines()

stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize)

# similarity with cosine_smiliarity function
def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

#print (cosine_sim('করোনা কেড়ে নিলো দেড় লাখ প্রাণ', 'করোনা ভাইরাস কেড়ে নিলো দেড় লাখ প্রাণ'))
size = len(data.index)
for i in range(1,size):
  news = data.loc[i,:]
  print("Similarities with recent news article :",i,"-",cosine_sim(str(input_text), str(news)))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Similarities with recent news article : 1 - 0.46041014644128897
Similarities with recent news article : 2 - 0.40840048351894886
Similarities with recent news article : 3 - 0.46098477241782465
Similarities with recent news article : 4 - 0.5070299872061635
Similarities with recent news article : 5 - 0.33408351047262747
Similarities with recent news article : 6 - 0.39244259912930424
Similarities with recent news article : 7 - 0.38143986127427953
Similarities with recent news article : 8 - 0.4459587206098994
Similarities with recent news article : 9 - 0.46041014644128897
Similarities with recent news article : 10 - 0.4604101464412889
Similarities with recent news article : 11 - 0.48995323531300083
Similarities with recent news article : 12 - 0.40840048351894886
Similarities with recent news article : 13 - 0.39244259912930424
Similarities with recent news article : 14 - 0.43231230509999